In [1]:
from timeit import default_timer as timer
import math

import numpy as np
import matplotlib.pyplot as plt

def PCsPattern(size,radius,a):
    '''create sites for PCs.'''
    h=a*np.sin(np.pi/3) #space between 2 holes in y line
    nx=int((size-2*radius)/a);Lx=a*nx+2*radius
    ny=int((size-2*radius)/h);Ly=h*ny+2*radius
    pattern = []
    for i in range(ny+1):
        y = i*h - Ly/2 + radius
        for j in range(nx+1-(i%2)):
            x = (j + (i%2)*0.5)*a -Lx/2 + radius
            pattern.append([x,y])
    return np.array(pattern)

def raithDots(p=1000,start_dose=1,dose_step=0.1,nx=30,ny=70):
    '''Raith Demo dots dose test: pitch=1 (default) or 0.5 µm.'''
    pattern = np.zeros_like(np.ndarray(((nx+1)*(ny+1),3)))
    for i in range(pattern.shape[0]):
        x = -nx*p/2 + (i%(nx+1))*p
        stepy=int(i/(nx+1))
        y = -ny*p/2 + stepy*p
        dose = start_dose + stepy*dose_step
        pattern[i]=[x,y,dose]
    return pattern

def dotPattern(x,y,dose=1):
    '''just dots and dose.'''
    return np.array([[x,y,dose]])

def linPattern(x1,y1,x2,y2,ss,dose=1):
    '''line start at (x1,y2), end at (x2,y2), step and dose at each step.'''
    d=np.sqrt((x2-x1)**2+(y2-y1)**2)
    a=np.arctan((y2-y1)/(x2-x1))
    pattern = dotPattern(x1,y1,dose)
    for s in np.arange(ss,d+ss,ss):
        pattern = np.append(pattern, dotPattern(x1+s*np.cos(a),y1+s*np.sin(a),dose), axis=0)
    return pattern

#l=linPattern(0,0,100,0,4)

def rectPattern(x1,y1,x2,y2,ss,dose=1):
    '''rectangle = rows x lines.'''
    pattern=linPattern(x1,y1,x2,y1,ss,dose)
    for h in np.arange(ss,y2+ss,ss):
        pattern=np.append(pattern,linPattern(x1,y1+h,x2,y1+h,ss,dose),axis=0)
    return pattern

#rect=rectPattern(0,0,20,10,3)

def ringPattern(x,y,r,ss,dose=1):
    '''ring center (x,y), radius r, step size and dose.'''
    pattern = dotPattern(x+r,y,dose)
    try:
        n=int(2*np.pi*r/ss)
        a=2*np.pi/n
        for ai in np.arange(a,2*np.pi,a):
            pattern=np.append(pattern,dotPattern(x+r*np.cos(ai),y+r*np.sin(ai),dose),axis=0)
    except ZeroDivisionError:
        n=0
    return pattern

#ring=ringPattern(0,0,r,4)

def circPattern(x,y,r,ss,dose=1):
    '''circle filled center at (x,y), radius, step size and dose.'''
    pattern=ringPattern(x,y,0,ss,dose)
    try:
        nr=int(r/ss)
        for i in range(1,nr):
            pattern=np.append(pattern,ringPattern(x,y,i*ss,ss,dose),axis=0)
    except ZeroDivisionError:
        nr=0
    return pattern

#circ=circPattern(0,0,r,ss)

def trianPattern(x,y,a,ss,dose=1):
    '''todo!'''
    return

def polyPattern(points,ss,dose=1):
    '''todo!'''
    return

def rot(alpha):
    '''rotation angle alpha.
    use case:
        l=linPattern(0,0,36,0,4)
        rota=rot(np.pi/3)
        l2=l[:,:-1]*rota'''
    return np.matrix( [[np.cos(alpha),-np.sin(alpha)],[np.sin(alpha),np.cos(alpha)]] )

def mvPattern(x,y,pattern):
    '''move pattern to (x,y).'''
    pout=np.zeros_like(pattern)
    for i in range(pattern.shape[0]):
        pout[i,:-1]=[x,y]-pattern[i,:-1]
        pout[i,-1]=pattern[i,-1]
    return pout

def replacePointsByPattern(points, pattern):
    '''map pattern to each site of points.'''
    pout=mvPattern(points[0,0],points[0,1],pattern)
    for point in points[1:]:
        pout=np.append(pout,mvPattern(point[0],point[1],pattern),axis=0)
    return pout


In [ ]:
p1 = mvPattern(25000.,0.,raithDots(500,0.02,0.02,15,140))
p2 = mvPattern(-25000.,0.,raithDots())
plt.plot(p2[:,0], p2[:,1], 'o', ms=2)
plt.plot(p1[:,0], p1[:,1], 'o', ms=1)
plt.axis('equal');plt.show()

In [ ]:
p2=mvPattern(-25000.,0.,p2)
plt.plot(p2[:,0], p2[:,1], 'o', ms=2)
plt.axis('equal');plt.show()

In [2]:
# create PC sites:
L=4000.0;r=38.0;a=170.0;ss=4
start = timer()
site_points=PCsPattern(L,r,a)
pattern=circPattern(0,0,r,ss)
#pattern=dotPattern(0,0)
p3=replacePointsByPattern(site_points,pattern)
dt = timer() - start
print("total {} points created in {} s".format(p3.shape[0], dt))

total 142240 points created in 1.746003098557464 s


In [ ]:
site_points.shape

site_points.shape,pattern.shape
#dotPattern(0.,0.)[0],pattern[0]

l=linPattern(0,0,36,0,4);l.shape;rota=rot(np.pi/3);l2=l[:,:-1]*rota

plt.plot(l[:,0], l[:,1], 'o', ms=1)
plt.plot(l2[:,0], l2[:,1], 'o', ms=1)
plt.axis('equal');plt.show()

In [ ]:
plt.plot(site_points[:,0], site_points[:,1], 'o', ms=1)
plt.axis('equal');plt.show()

In [ ]:
plt.plot(pattern[:,0], pattern[:,1], 'o', ms=1)
plt.axis('equal');plt.show()

In [ ]:
#p2=dotPattern(0,0)
plt.plot(p3[:,0], p3[:,1], 'o', ms=1)
plt.axis('equal');plt.show()

## SEM parameters

In [3]:
from scipy.constants import e

Nesim=1e5 #number electron paths
PGSpeed=6e6 #pattern generator max frequency in Hz
semCurrentsAt30kV=np.array([24,48,196,411,1780,7300])*1e-12 #SEM currents tab in Ampere at 30 kV
minimalDoses=semCurrentsAt30kV/PGSpeed #in C
Nelectrons=minimalDoses/e
normalizeFactors=Nelectrons/Nesim

In [ ]:
normalizeFactors

In [4]:
import os
#os.chdir('d:/labo/projets/FlatLight/casino3')
os.chdir('c:/Users/Loon/Documents/jupyter/ebl')
import Casino3 as cs3
#from scipy.interpolate import interp1d
#from scipy.stats import threshold
#os.chdir('e:/Luan/boulot/CNRS/CRHEA/FlatLight/casino/data')
os.chdir('e:/dev/ebl/cs3/data')
filename='cs3_30kV_1e5_xy800_z50.dat'
sim=cs3.Casino3(filename);zi=25
mcdata=np.c_[sim.rrange,sim.data[zi]*normalizeFactors[2]]
sim.data.shape,np.min(mcdata[:,1]),np.max(mcdata[:,1])

((50, 800), 0.0, 1.1602830232319248)

In [ ]:
sim.data.shape

In [ ]:
cs3.plotSim(sim,[5,10,15,20,25,30,35,40,45])

In [ ]:
filename='cs3_30kV_1e5_xy800_z50_Si.dat'
simsi=cs3.Casino3(filename)
mcdatasi=np.c_[simsi.rrange,simsi.data[zi]*normalizeFactors[2]]
simsi.data.shape,np.min(mcdatasi[:,1]),np.max(mcdatasi[:,1])

## Fit 3 gauss:
sur cs3_30kV_1e5_xy800_z50_Si.dat

In [5]:
def pyQtpsf(x_in):
    temp = 0.0
    # coefficients
    a = 8.9525600038156994E-01
    b = 1.6686131356451853E-01
    c = 1.1206358623797494E+00
    d = -5.1309230854665384E+03
    f = -8.6370673361282471E+00
    g = 1.7726195110992959E+00
    h = 4.5932595024587547E-02
    i = 3.0256646690173705E-02
    j = 1.7543824562579575E+10
    k = -1.0485715178472005E+03
    l = 2.0552478426806061E+02
    Offset = 1.1088144123474852E-03
    temp = a * math.exp(-b * x_in) + c * math.exp(-1.0 * (x_in-d) * (x_in-d) / (f * f)) + g * math.exp(-1.0 * (x_in-h) * (x_in-h) / (i * i)) + j * math.exp(-1.0 * (x_in-k) * (x_in-k) / (l * l))
    temp += Offset
    return temp

In [ ]:
x=np.linspace(np.min(mcdatasi[:,0]), np.max(mcdatasi[:,0]),simsi.nr)
y=np.array(list(map(pyQtpsf,x)))
fitdatasi=np.c_[x,y]
np.min(fitdatasi[:,1]), np.max(fitdatasi[:,1])

In [ ]:
plt.semilogy(mcdatasi[:,0],mcdatasi[:,1],label=simsi.zlabels[zi]+'_Si')
plt.semilogy(fitdatasi[:,0],fitdatasi[:,1],label='Fit3exps_psf_Si')
plt.title('Energy Density Distribution')
plt.xlabel('r[nm]')
plt.ylabel('[eV]')
plt.legend(loc='upper right', shadow=True)
plt.show()

In [ ]:
#cs3.plotSim(sim,[1])
plt.semilogy(mcdata[:,0],mcdata[:,1],label=sim.zlabels[zi])
plt.semilogy(mcdatasi[:,0],mcdatasi[:,1],label=simsi.zlabels[zi]+'_Si')
plt.title('Energy Density Distribution')
plt.xlabel('r[nm]')
plt.ylabel('[eV]')
plt.legend(loc='upper right', shadow=True)
plt.show()

In [ ]:
def coulombToElectron(c):
    return c/e

coulombToElectron(130e-6)

### Naive Python

In [6]:
def myinterp1d(r,sr,M,data):
    r_i = r/sr
    #i_lo=int(np.floor(r_i));i_hi=int(np.ceil(r_i))
    i_lo=int(r_i);i_hi = i_lo+1 if r_i > i_lo else i_lo
    if (i_hi == i_lo or i_hi == M):
        return data[i_hi,1]
    else:
        slope = (data[i_hi,1]-data[i_lo,1])/(data[i_hi,0]-data[i_lo,0])
        return slope*(r-data[i_lo,0]) + data[i_lo,1]

def expose1(x,y,pattern,sr,M,dosedata):
    re=0
    for point in pattern:
        r=np.sqrt((x-point[0])**2+(y-point[1])**2)
        re += myinterp1d(r,sr,M,dosedata)
    return re

def expose2(x,y,pattern):
    #re=0
    for point in pattern:
        idx=np.sqrt((x-point[0])**2+(y-point[1])**2)/ss
        idx=list(map(int,r/ss))
        #re += z[int(r/ss)]
    return idx

myinterp1d(6.5,sim.sr,sim.nr,dosedata),expose1(6.5,100,pattern,sim.sr,sim.nr,dosedata)

for point in p2:
    r2=np.sqrt((grid_x-point[0])**2+(grid_y-point[1])**2)

from numba import vectorize
@vectorize(['float32(float32, float32)'], target='cuda')
def Add(a, b):
    return a + b

# Initialize arrays
N = 100000
A = np.ones(N, dtype=np.float32)
B = np.ones(A.shape, dtype=A.dtype)
C = np.empty_like(A, dtype=A.dtype)

# Add arrays on GPU
C = Add(A, B)

r2=r2/ss
r2[0,0]

grid_x,grid_y=np.mgrid[-WF/2:WF/2:NP*1j,-WF/2:WF/2:NP*1j]
start = timer()
Z=expose1(grid_x,grid_y,pattern)
dt = timer() - start
print("Exposure created in %f s" % dt)

th=threshold(Z, 5e7)
a=cs3.np.clip(th,0,1)
cs3.plt.imshow(a, extent=(-WF/2,WF/2,-WF/2,WF/2), origin='center')
cs3.plt.show()

point=dotPattern(0,0)
grid_x,grid_y=np.mgrid[-WF:WF:NP*2j,-WF:WF:NP*2j]
start = timer()
PSF=expose(grid_x,grid_y,point)
dt = timer() - start
print("Exposure created in %f s" % dt)
cs3.plt.imshow(PSF, extent=(-WF,WF,-WF,WF), origin='center')
cs3.plt.show()

## Faster Execution with Numba

def foosum(x,y,points,fct):
    re=0
    for point in points:
        r=cs3.np.sqrt((x-point[0])**2+(y-point[1])**2)
        re += fct(r)
    return re

def expose(x,y,pattern):
    re=0
    for point in pattern:
        r=np.sqrt((x-point[0])**2+(y-point[1])**2)
        re += f(r)
    return re


from numba import autojit

@autojit
def psf(r):
    return f(np.abs(r))

@autojit
def expose1(x,y,pattern):
    re=0
    for point in pattern:
        r=np.sqrt((x-point[0])**2+(y-point[1])**2)
        re += psf(r)
    return re

start = timer()
Z=expose1(grid_x,grid_y,pattern)
dt = timer() - start
print("Exposure created in %f s" % dt)

## Even Bigger Speedups with CUDA Python
For 2GB of RAM on NVIDIA Quadro K620, i use 1.5GB that leads to NP = 7872 pixels.

In [7]:
from numba import cuda
from numba import *

interp1d_gpu = cuda.jit(device=True)(myinterp1d)
pyQtpsf_gpu = cuda.jit(device=True)(pyQtpsf)

In [ ]:
def ramGB(n,sof=8):
    return sof*2.048*n**2/2147483648

def pxls(ramGB,sof=8):
    return np.sqrt(ramGB*2147483648/2.048/sof)

def compute_pixelSize(WF=100000,NP=7872):
    return np.float32(WF/(NP-1))

compute_pixelSize(5000)

In [8]:
@cuda.jit
def expose_kernel2(dest,point,NP,WF,pixel_size,sr,nr,dosedata):
    '''expose with dose factor'''
    #height = 512; WF = 4000.0;pixel_size = 7.8277888
    startX, startY = cuda.grid(2)
    gridX = cuda.gridDim.x * cuda.blockDim.x
    gridY = cuda.gridDim.y * cuda.blockDim.y
    for x in range(startX, NP, gridX):
        val_x = -WF/2 + x*pixel_size
        for y in range(startY, NP, gridY):
            val_y = -WF/2 + y*pixel_size
            r = math.sqrt((val_x-point[0])**2+(val_y-point[1])**2)
            dest[y, x] += point[2]*interp1d_gpu(r,sr,nr,dosedata)

In [9]:
NP=576;WF=5000.0 #38232 dots in 394 s on i3 core + 830M
#NP=7872;WF=5000.0 in 59793.558081 s on i5 core + K620
blockdim = (32, 2)
griddim = (int(NP/blockdim[0]),int(NP/blockdim[1]))
pixel_size=np.float32(WF/(NP-1))
z = np.zeros((NP, NP), dtype = np.float64)

start = timer()
d_image = cuda.to_device(z)
d_mcdata = cuda.to_device(mcdata)
for point in p3:
    expose_kernel2[griddim, blockdim](d_image,point,NP,WF,pixel_size,sim.sr,sim.nr,d_mcdata) 

d_image.to_host()
d_mcdata.to_host()
dt = timer() - start
print("eblWF created on GPU in %f s" % dt)

eblWF created on GPU in 388.444244 s


In [10]:
filename='PCs_WF5u_circle_norm_PC2_576px'
#filename='PCs_WF5u_circle_norm_PC2_Si_576px'
np.savetxt(filename,z)
#z=np.loadtxt(filename)

In [ ]:
filename='PCs_WF5u_circle_norm_PC2_576px'
z=np.loadtxt(filename)

In [ ]:
plt.imshow(z,origin='lower', extent=[-WF/2, WF/2, -WF/2, WF/2])
plt.show()

In [ ]:
@cuda.jit
def expose_fit_kernel2(dest,point,NP,WF,pixel_size,sr,nr,dosedata):
    '''expose with dose factor'''
    #height = 512; WF = 4000.0;pixel_size = 7.8277888
    startX, startY = cuda.grid(2)
    gridX = cuda.gridDim.x * cuda.blockDim.x
    gridY = cuda.gridDim.y * cuda.blockDim.y
    for x in range(startX, NP, gridX):
        val_x = -WF/2 + x*pixel_size
        for y in range(startY, NP, gridY):
            val_y = -WF/2 + y*pixel_size
            r = math.sqrt((val_x-point[0])**2+(val_y-point[1])**2)
            dest[y, x] += point[2]*pyQtpsf_gpu(r)

In [ ]:
NP=576;WF=5000.0 #38232 dots in 394 s on i3 core + 830M
#NP=7872;WF=5000.0 in 59793.558081 s on i5 core + K620
blockdim = (32, 2)
griddim = (int(NP/blockdim[0]),int(NP/blockdim[1]))
pixel_size=np.float32(WF/(NP-1))
z = np.zeros((NP, NP), dtype = np.float64)

start = timer()
d_image = cuda.to_device(z)
d_fitdata = cuda.to_device(fitdatasi)
for point in p3:
    expose_fit_kernel2[griddim, blockdim](d_image,point,NP,WF,pixel_size,sim.sr,sim.nr,d_fitdata) 

d_image.to_host()
d_fitdata.to_host()
dt = timer() - start
print("eblWF created on GPU in %f s" % dt)

In [ ]:
filename='PCs_WF5u_circle_fitPSF_PC2_Si_576px'
np.savetxt(filename,z)
#z=np.loadtxt(filename)

In [ ]:
#wfimage *= normalizeFactors[0]
plt.imshow(z,origin='lower', extent=[-WF/2, WF/2, -WF/2, WF/2])
plt.show()

In [ ]:
a=np.clip(th,0,1)

@cuda.jit
def expose_kernel(dest,point,sr,nr,mcdata):
    height = dest.shape[0]
    width = dest.shape[1]
    startX, startY = cuda.grid(2)
    gridX = cuda.gridDim.x * cuda.blockDim.x
    gridY = cuda.gridDim.y * cuda.blockDim.y
    for x in range(startX, width, gridX):
        for y in range(startY, height, gridY):
            r = math.sqrt((x-point[0])**2+(y-point[1])**2)
            dest[y, x] += interp1d_gpu(r,sr,nr,mcdata)

NP=576;WF=4000.0
wfimage = np.zeros((NP, NP), dtype = np.float64)
blockdim = (32, 3)
griddim = (int(NP/blockdim[0]),int(NP/blockdim[1]))

start = timer()
d_image = cuda.to_device(wfimage)
for point in p3:
    expose_kernel[griddim, blockdim](d_image,point,sim.sr,sim.nr,mcdata) 
d_image.to_host()
dt = timer() - start
print("eblWF created on GPU in %f s" % dt)


plt.imshow(wfimage,origin='center', extent=[-WF/2, WF/2, -WF/2, WF/2])
plt.show()

np.max(wfimage)

moyenne=np.mean(wfimage)
np.max(wfimage), np.min(wfimage), sim.maxED, moyenne

from scipy.stats import threshold
th=threshold(wfimage, 1.14*moyenne)
a=np.clip(th,0,1)
imshow(a, extent=(-WF/2, WF/2, -WF/2, WF/2), origin='center')
show()


In [ ]:
@cuda.jit
def expose_kernel3(dest,point,pixel_size,sr,nr,dosedata,NP,WF,WFcenter):
    '''expose with dose factor'''
    #height = 512; WF = 4000.0;pixel_size = 7.8277888
    startX, startY = cuda.grid(2)
    gridX = cuda.gridDim.x * cuda.blockDim.x
    gridY = cuda.gridDim.y * cuda.blockDim.y
    for ix in range(startX, NP, gridX):
        val_x = -WF/2 + WFcenter[0] + ix*pixel_size
        for iy in range(startY, NP, gridY):
            val_y = -WF/2 + WFcenter[1] + iy*pixel_size
            r = math.sqrt((val_x-point[0])**2+(val_y-point[1])**2)
            dest[iy, ix] += point[2]*interp1d_gpu(r,sr,nr,dosedata)

In [ ]:
@cuda.jit
def expose_kernel4(dest,point,pixel_size,sr,nr,dosedata,NP,min_x, min_y):
    '''expose with dose factor'''
    #height = 512; WF = 4000.0;pixel_size = 7.8277888
    startX, startY = cuda.grid(2)
    gridX = cuda.gridDim.x * cuda.blockDim.x
    gridY = cuda.gridDim.y * cuda.blockDim.y
    for ix in range(startX, NP, gridX):
        val_x = min_x + ix*pixel_size
        for iy in range(startY, NP, gridY):
            val_y = min_y + iy*pixel_size
            r = math.sqrt((val_x-point[0])**2+(val_y-point[1])**2)
            dest[iy, ix] += point[2]*interp1d_gpu(r,sr,nr,dosedata)

## Kernel4 run

In [ ]:
#center=[-13500.,33500.]
#minx,miny=-16000,31000
NP=576;WF=1000.0
minX,minY=-15500,-35500
blockdim = (32, 3)
griddim = (int(NP/blockdim[0]),int(NP/blockdim[1]))
pixel_size=np.float32(WF/(NP-1))
wfimage = np.zeros((NP, NP), dtype = np.float64)

l=0;c=0;minx=minX+c*WF;miny=minY+l*WF
filename='expose_30kV_Dots1_'+str(l)+'_'+str(c)
start = timer()
d_image = cuda.to_device(wfimage)
d_mcdata = cuda.to_device(mcdata)
for point in p3:
    expose_kernel4[griddim, blockdim](d_image,point,pixel_size,sim.sr,sim.nr,d_mcdata,NP,minx,miny) 

d_image.to_host()
d_mcdata.to_host()
dt = timer() - start
print("eblWF created on GPU in %f s" % dt)
#np.savetxt(filename,wfimage)

In [ ]:
np.min(wfimage),np.max(wfimage),pixel_size

In [ ]:
from pylab import imshow, show
imshow(wfimage,origin='lower', extent=[minx, minx+WF, miny, miny+WF])
show()

In [ ]:
from scipy.stats import threshold
th=threshold(wfimage, sim.maxED/10)
a=np.clip(th,0,1)
imshow(a, extent=(minx, minx+WF, miny, miny+WF), origin='center')
show()

### kernel4 scan run on raith doses 1

In [ ]:
NP=1056;WF=1000.0
minX,minY=-15500,-35500
blockdim = (32, 6)
griddim = (int(NP/blockdim[0]),int(NP/blockdim[1]))
#pixel_size=np.float32(WF/(NP-1))
pixel_size=compute_pixelSize(WF,NP)
for col in range(31):
    minx=minX + col*WF
    for raw in range(12,24):
        filename='expose_30kV_Dots1_'+str(raw)+'_'+str(col)
        miny = minY + raw*WF
        wfimage = np.zeros((NP, NP), dtype = np.float64)
        start = timer()
        d_image = cuda.to_device(wfimage)
        d_dosedata = cuda.to_device(dosedata)
        for point in p2:
            expose_kernel4[griddim, blockdim](d_image,point,pixel_size,sim.sr,sim.nr,d_dosedata,NP,minx,miny) 
        d_image.to_host()
        d_dosedata.to_host()
        dt = timer() - start
        print("eblWF created on GPU in %f s" % dt)

## Kernel3 run
(dest,point,pixel_size,sr,nr,dosedata,NP,WF,WFcenter):

In [ ]:
NP=576;WF=5000.0
center=[-13500.,33500.]
#minx,miny=-13500,33500
pixel_size=np.float32(WF/(NP-1))
wfimage = np.zeros((NP, NP), dtype = np.float64)
blockdim = (32, 3)
griddim = (int(NP/blockdim[0]),int(NP/blockdim[1]))

start = timer()
d_image = cuda.to_device(wfimage)
d_dosedata = cuda.to_device(dosedata)
for point in p1:
    expose_kernel3[griddim, blockdim](d_image,point,pixel_size,sim.sr,sim.nr,d_dosedata,NP,WF,center)
    #pass

d_image.to_host()
d_dosedata.to_host()
dt = timer() - start
print("eblWF created on GPU in %f s" % dt)

## Kernel2 run

In [ ]:
NP=1056;WF=1000.0
minX,minY=-15500,-35500
blockdim = (32, 3)
griddim = (int(NP/blockdim[0]),int(NP/blockdim[1]))
pixel_size=np.float32(WF/(NP-1))
wfimage = np.zeros((NP, NP), dtype = np.float64)

l=35;c=15;minx=minX+c*WF;miny=minY+l*WF
filename='expose_30kV_Dots1_'+str(l)+'_'+str(c)
start = timer()
d_image = cuda.to_device(wfimage)
d_dosedata = cuda.to_device(dosedata)
for point in p3:
    expose_kernel2[griddim, blockdim](d_image,point,NP,WF,pixel_size,sim.sr,sim.nr,d_dosedata) 

d_image.to_host()
d_dosedata.to_host()
dt = timer() - start
print("eblWF created on GPU in %f s" % dt)

In [ ]:
minX,minY

In [ ]:
#from pylab import imshow, show
imshow(wfimage,origin='lower', extent=[minX, minX+WF, minY, minY+WF])
show()

In [ ]:
NP=480;WF=100000.0
pixel_size=np.float32(WF/(NP-1))
wfimage = np.zeros((NP, NP), dtype = np.float64)
blockdim = (32, 6)
griddim = (15, 80)

In [ ]:
start = timer()
d_image = cuda.to_device(wfimage)
d_dosedata = cuda.to_device(dosedata)
for idx in range(0,200):
    expose_kernel2[griddim, blockdim](d_image,p2[idx],NP,WF,pixel_size,sim.sr,sim.nr,d_dosedata) 
d_image.to_host()
d_dosedata.to_host()
dt = timer() - start
print("eblWF created on GPU in %f s" % dt)

In [ ]:
np.savetxt('expose_30kV_raithDots1_c',wfimage)

In [ ]:
p2[[0,-1]]

In [ ]:
# plus grosse en 1.5 Go:
NP=7872;WF=5000.0
pixel_size=np.float32(WF/(NP-1))
#wfimage=np.loadtxt('expose_30kV_100')
blockdim = (32, 6)
griddim = (246, 1312)

start = timer()
d_image = cuda.to_device(wfimage)
d_dosedata = cuda.to_device(dosedata)
for idx in range(30000,145152):
    expose_kernel2[griddim, blockdim](d_image,p2[idx,:2],NP,WF,pixel_size,sim.sr,sim.nr,d_dosedata) 
d_image.to_host()
d_dosedata.to_host()
dt = timer() - start
print("eblWF created on GPU in %f s" % dt)

In [ ]:
p2[[0,-1]]

In [ ]:
NP=1056;WF=1000.0
minX,minY=-15500,-35500

In [ ]:
l=0;c=0;minx=minX+c*WF;miny=minY+l*WF
filename='expose_30kV_Dots1_'+str(l)+'_'+str(c)
wfimage=np.loadtxt(filename)

In [ ]:
from pylab import imshow, show
imshow(wfimage,origin='lower', extent=[minx, minx+WF, miny, miny+WF])
show()
#imshow(wfimage,origin='lower', extent=[-WF/2, WF/2, -WF/2, WF/2])
#show()

In [ ]:
np.min(wfimage),np.max(wfimage),pixel_size

In [ ]:
from scipy.stats import threshold
th=threshold(wfimage, sim.maxED/20)
a=np.clip(th,0,1)
imshow(a, extent=(minx, minx+WF, miny, miny+WF), origin='center')
show()


In [ ]:
import scipy.misc
scipy.misc.imsave('outfile.jpg', wfimage)

In [ ]:
bpp=1512900000/(NP**2)
RAM=64;np.sqrt(RAM*1.024e9/bpp)

In [ ]:
81810/96

In [ ]:
96*852

In [ ]:
from scipy.constants import e

In [ ]:
D0=24e-12/6e6;Ne=D0/e;Norm=Ne*1e-6
#wfimage *= Norm

In [ ]:
np.max(wfimage), np.min(wfimage)

In [ ]:
[D0,Ne,Norm]

# use PyQtGraph:

In [ ]:
from pyqtgraph.Qt import QtCore, QtGui
import pyqtgraph as pg
import pyqtgraph.opengl as gl
import sys

## Create a GL View widget to display data
app = QtGui.QApplication([])

In [ ]:
NP=576;WF=5000.0 #38232 dots in 394 s on i3 core + 830M
#NP=7872;WF=5000.0 in 59793.558081 s on i5 core + K620
blockdim = (32, 2)
griddim = (int(NP/blockdim[0]),int(NP/blockdim[1]))

In [ ]:
filename='PCs_WF5u_circle_norm_PC4_576px'
mcdata=np.c_[sim.rrange,sim.data[zi]*normalizeFactors[4]]
z = np.zeros((NP, NP), dtype = np.float64)

start = timer()
d_image = cuda.to_device(z)
d_mcdata = cuda.to_device(mcdata)
for point in p3:
    expose_kernel2[griddim, blockdim](d_image,point,NP,WF,pixel_size,sim.sr,sim.nr,d_mcdata) 

d_image.to_host()
d_mcdata.to_host()
dt = timer() - start
print("eblWF created on GPU in %f s" % dt)
np.savetxt(filename,z)
x=np.linspace(-WF/2,WF/2,NP)
y=np.linspace(-WF/2,WF/2,NP)

In [ ]:
filename='PCs_WF5u_circle_norm_PC2_576px'
#filename='PCs_WF5u_circle_norm_PC2_Si_576px'
#filename='PCs_WF5u_circle_fitPSF_PC2_Si_576px'
z=np.loadtxt(filename)
x=np.linspace(-WF/2,WF/2,NP)
y=np.linspace(-WF/2,WF/2,NP)
np.min(z),np.max(z)

In [ ]:
2087.1376355893435/229.81964976152321,2572.6035667713772/283.2754489253856

In [ ]:
z *= 9
np.min(z),np.max(z)

In [ ]:
w = gl.GLViewWidget()
w.show()
w.setWindowTitle('GLSurfacePlot: '+filename)
w.setCameraPosition(distance=50)

## Add a grid to the view
g = gl.GLGridItem()
g.scale(1,1,1)
g.setDepthValue(1)  # draw grid after surfaces since they may be translucent
w.addItem(g)


## Simple surface plot example
## x, y values are not specified, so assumed to be 0:50
p1 = gl.GLSurfacePlotItem(x=x,y=y,z=z, shader='shaded', color=(0.5, 0.5, 1, 1))
p1.translate(0, 0, -np.mean(z))
w.addItem(p1)

if (sys.flags.interactive != 1) or not hasattr(QtCore, 'PYQT_VERSION'):
    QtGui.QApplication.instance().exec_()


In [ ]:
np.min(z)

In [ ]:
mM=[(28.141181603451873, 34.686789664332828),
    (56.282363206903746, 69.373579328665656),
    (229.81964976152321, 283.2754489253856),
    (481.91773495911178, 594.01127300168048),
    (2087.1376355893435, 2572.6035667713772),
    (8559.6094043832582, 10550.565189567757),]

In [ ]:
l=[]
for i in range(4):
    l.append([semCurrentsAt30kV[i+1]/semCurrentsAt30kV[i], mM[i+1][0]/mM[i][0],mM[i+1][1]/mM[i][1]])
l